In [16]:
from bs4 import BeautifulSoup
import json

# Load your saved Yelp page
with open("listing1.html", "r", encoding="utf-8") as f:
    soup = BeautifulSoup(f, "html.parser")

# Business-level info
business_name = soup.select_one(
    "div.photo-header-content__09f24__q7rNO div.headingLight__09f24__N86u1"
)
business_name_text = business_name.get_text(strip=True) if business_name else None

overall_rating = soup.select_one("span.y-css-1ms5w5p")
if overall_rating:
    overall_rating_text = overall_rating.get("aria-label", overall_rating.get_text(strip=True))
else:
    overall_rating_text = None

review_count = soup.select_one("span.y-css-1q46f5r")
review_count_text = review_count.get_text(strip=True) if review_count else None

# Reviews
reviews = []
review_items = soup.select("#reviews > section > div.y-css-mhg9c5 > ul > li")

# Skip the last review with [:-1]
for item in review_items[:-1]:
    review = {}

    # Add business-level info to each review
    review["business_name"] = business_name_text
    review["overall_rating"] = overall_rating_text
    review["total_review_count"] = review_count_text

    # Reviewer name
    name = item.select_one("div.user-passport-info span")
    review["reviewer_name"] = name.get_text(strip=True) if name else None

    # Reviewer location (directly target location container)
    location_span = item.select_one("div.user-passport-info div.y-css-14zpyii > span")
    review["reviewer_location"] = location_span.get_text(strip=True) if location_span else None

    # Review date
    date = item.select_one("div.y-css-scqtta")
    review["date"] = date.get_text(strip=True) if date else None

    # Review text
    comment = item.select_one("p span")
    review["text"] = comment.get_text(strip=True) if comment else None

    reviews.append(review)

# Save results to JSON
with open("parsed.json", "w", encoding="utf-8") as f:
    json.dump(reviews, f, indent=2, ensure_ascii=False)

print(" Scraping done! Data saved to parsed.json")


 Scraping done! Data saved to parsed.json


In [24]:
from bs4 import BeautifulSoup
import json
import os

# List of files to process
file_names = ["page1.html", "page2.html", "page3.html"]
all_reviews = []

# --- Business-level info (extracted once from the first page) ---
business_name_text = None
overall_rating_text = None
review_count_text = None

first_file_path = file_names[0]
if os.path.exists(first_file_path):
    with open(first_file_path, "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "html.parser")

        # Selectors for business info
        business_name = soup.select_one("div.photo-header-content__09f24__q7rNO div.headingLight__09f24__N86u1")
        overall_rating = soup.select_one("span.y-css-1ms5w5p")
        review_count = soup.select_one("span.y-css-1q46f5r")

        business_name_text = business_name.get_text(strip=True) if business_name else None
        overall_rating_text = overall_rating.get("aria-label", overall_rating.get_text(strip=True)) if overall_rating else None
        review_count_text = review_count.get_text(strip=True) if review_count else None

# --- Reviews (loop through all pages) ---
for file_name in file_names:
    if not os.path.exists(file_name):
        print(f"Warning: File not found: {file_name}. Skipping...")
        continue

    with open(file_name, "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "html.parser")

    review_items = soup.select("#reviews > section > div.y-css-mhg9c5 > ul > li")

    # Skip the last review, which is often a pagination link or a different format
    for item in review_items[:-1]:
        review = {}

        # Add business-level info to each review
        review["business_name"] = business_name_text
        review["overall_rating"] = overall_rating_text
        review["total_review_count"] = review_count_text

        # Reviewer name
        name = item.select_one("div.user-passport-info span")
        review["reviewer_name"] = name.get_text(strip=True) if name else None

        # Reviewer location
        location_span = item.select_one("div.user-passport-info div.y-css-14zpyii > span")
        review["reviewer_location"] = location_span.get_text(strip=True) if location_span else None

        # Review date
        date = item.select_one("div.y-css-scqtta")
        review["date"] = date.get_text(strip=True) if date else None

        # Review text
        comment = item.select_one("p span")
        review["text"] = comment.get_text(strip=True) if comment else None

        all_reviews.append(review)

# Save the combined results to the new JSON file
with open("multi_page_sparcer.json", "w", encoding="utf-8") as f:
    json.dump(all_reviews, f, indent=2, ensure_ascii=False)

print("Scraping done! Data for all pages saved to multi_page_sparcer.json")

Scraping done! Data for all pages saved to multi_page_sparcer.json
